# Machine Learning Chatbot

In [1]:
# Reference:
# **********************************************************************************************************
#    Title: Build Your Own Chatbot Using Deep Learning
#    Author: Sreekanth
#    Date: Oct 17, 2021
#    Availability: https://medium.com/@rr_42830/build-your-own-chatbot-using-deep-learning-23a022638067
#
# **********************************************************************************************************
!pip install fuzzywuzzy
!pip install python-Levenshtein

import os
import json
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
# Preprocessing and modelling
def model(data):
    # extract data from json file
    training_sentences = []
    training_labels = []
    labels = []
    responses = []


    for intent in data['intents']:
        for pattern in intent['patterns']:
            training_sentences.append(pattern)
            training_labels.append(intent['tag'])
        responses.append(intent['responses'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

    num_classes = len(labels)

    # one hot encoding
    lbl_encoder = LabelEncoder() # convert categorical variables to numerical labels ("greeting", "bye", "thanks") -> (0, 1, 2)
    lbl_encoder.fit(training_labels) 
    training_labels = lbl_encoder.transform(training_labels)
    training_labels = keras.utils.to_categorical(training_labels, num_classes=num_classes)

    vocab_size = 1000
    embedding_dim = 20
    max_len = 20
    oov_token = "<OOV>" # out of vocabulary token value

    # tokenizing text, converting words to lowercase, filtering out punctuation, and converting text into sequences of integers
    tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) 
    tokenizer.fit_on_texts(training_sentences)
    word_index = tokenizer.word_index
    sequences = tokenizer.texts_to_sequences(training_sentences)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
    
    train_data = padded_sequences
    train_labels = training_labels
    
    # LSTM model
    model = keras.Sequential([
        Embedding(len(word_index)+1, embedding_dim, input_length=max_len),
        LSTM(embedding_dim),
        Dense(num_classes, activation="softmax")
    ])

    model.compile(
        optimizer="rmsprop",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(train_data, train_labels, epochs=150, verbose=0)
    
    return model, lbl_encoder, tokenizer

In [3]:
def openMenu():
    with open('Beverage_Menu.txt', 'r', encoding='utf-8') as file:
        for line in file:
            print(line)

In [4]:
# Name Entity Recognision (NER)
def NER(query):
    name_entity = []
    words =  nltk.word_tokenize(query)
    tagged = nltk.pos_tag(words)
    chunked = nltk.ne_chunk(tagged, binary=False)
    for chunk in chunked.leaves():
        if hasattr(chunk, 'label') or chunk[1] == 'NNP':
            name_entity.append(chunk[0])
                
    userName = ' ' + ' '.join(name for name in name_entity)
    return userName

In [5]:
def detectBeverage(query, menuPrice):
    from fuzzywuzzy import fuzz
    
    matching_items = []
    query = query.lower().split()
    for word in query:
        for item in menuPrice.keys():
            match_score = fuzz.token_set_ratio(word, item.lower()) # Calculate fuzzy match score between user query and coffee item
            if match_score > 80:
                if item not in matching_items:
                    matching_items.append(item.title())
    return matching_items

In [6]:
# detect number in user query
def detectNumber(query):
    num_dict = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six":6, "seven":7, "eight":8, "nine":9, "ten":10} # number dictionary
    
    quantity = []
    for word in query.split():
        if word.isdigit():
            quantity += [int(word)]
        else:
            for w in num_dict:
                if w == word:
                    quantity += [num_dict[word]]
    return quantity

In [7]:
# transaction function

def transaction(queryList, model2, tokenizer2, lbl_encoder2):
     
    menuPrice = {"Espresso": 1.50, "Americano": 1.50, "Cappuccino": 1.80, "Latte": 1.80, "Macchiato": 1.80, "Flat White": 2.10, "Mocha": 2.10, "Black Tea": 1.50}

    lastQuery = queryList[0]
    currentQuery = queryList[1]
    
    switch = True    
    while switch == True:
        # Detect quantity in user query
        temp_query = currentQuery
        quantity = detectNumber(temp_query)        
        
        # Detect beverage
        beverages = detectBeverage(currentQuery, menuPrice)
        if len(beverages) == 0:
            beverages = detectBeverage(lastQuery, menuPrice)
        
        if len(beverages) == 0:
            print('\nSorry, your order does not appear in our menu.')
            return True
        
        # Assume 1 for all beverage if no number detected
        if(len(quantity) == 0):
            for i in beverages:
                quantity += [1]   
    
        if len(quantity)!=0 and len(quantity) == len(beverages):
            # Find price of each beverage
            price = []
            for i, j in menuPrice.items():
                for k in beverages:
                    if k == i:
                        price += [j]

            #Calculate total price
            totalPrice = sum([p*q for p, q in zip(price, quantity)])

            print("\n{:<10} {:<15} {:<10}".format('Quantity', 'Beverage', 'Price'))
            print("----------------------------------\n")
            for i in range(len(price)):
                print("{:<10} {:<15} £{:<10.2f}".format(quantity[i], beverages[i], price[i]))
            print('\nTotal price is: £', "{:.2f}".format(totalPrice))
            flag = True
            while(flag == True):
                print('\nGroovy:\nGreat! Please confirm your item, quantity and total price (e.g. yes/no). If you want to exit transaction, type quit. :')
                confirmation = input().lower()
                prediction = model2.predict(pad_sequences(tokenizer2.texts_to_sequences([confirmation]), truncating='post', maxlen=20), verbose=0) # remove the query from the end when its length exceed max length (20)
                tag = lbl_encoder2.inverse_transform([np.argmax(prediction)]) #transform tag's numerical values back to string (0 -> "greeting")
                if ('quit' in confirmation):
                    flag = False
                    return False
                elif(tag == "positive"):
                    flag = False
                    print("\nThank you! Your order has been confirmed.")
                    return False
                else:
                    print('\nWould you like to reorder?')
                    reorder_query = input("\nUser:\n")
                    returnFlag = feedback(reorder_query, model2, tokenizer2, lbl_encoder2)
                    flag = False
                    return returnFlag
                           
            switch = False
        else:
            print("Groovy:\nPlease specify the quantity of each item (e.g. 2 mocha and 1 americano). If you want to exit transaction, type quit.")
            query = input("\nUser:\n").lower()
            if "quit" in query:
                return False
            else:
                lastQuery = currentQuery
                currentQuery = query


In [8]:
def feedback(query, model2, tokenizer2, lbl_encoder2):
    prediction = model2.predict(pad_sequences(tokenizer2.texts_to_sequences([query]), truncating='post', maxlen=20), verbose=0) # remove the query from the end when its length exceed max length (20)
    tag = lbl_encoder2.inverse_transform([np.argmax(prediction)]) #transform tag's numerical values back to string (0 -> "greeting")
    if (tag == "positive"):
        returnFlag = True
        return returnFlag
    elif (tag == "negative"):
        returnFlag = False
        return returnFlag

In [11]:
import random

def chat():
    with open('intent.json', encoding='utf-8') as file:
        data1 = json.load(file)
        
    with open('PositiveNegativeData.json', encoding='utf-8') as file1:
        data2 = json.load(file1)
    
    model1, lbl_encoder1, tokenizer1 = model(data1)
    model2, lbl_encoder2, tokenizer2 = model(data2)
    
    # parameters
    max_len = 20
    queryList = ["",""]
    newName = ""
    flag1 = True
    flag2 = False
    print("Groovy:\nHello, welcome to Chin's Coffee House! What is your name?")
    query = input("\nUser:\n")
    name = NER(query)
    while flag1:
        if flag2 == True:
            print('\nGroovy:\nWhat would you like to order? If you want to exit, type quit.')
            query = input("\nUser:\n")
            flag2 = False
        else:
            print('\nGroovy:\nHi'+ (newName if len(newName)!=0 else name) +". Welcome to Chin's Coffee House. How can I help you? If you want to exit, type quit.")
            query = input("\nUser:\n")
            
        if query.lower() == "quit":
            flag1 = False
            print("\nGroovy:\nThank you for visiting Chin's Coffee House. What do you think of my service?")
            query = input("\nUser:\n")
            prediction = feedback(query, model2, tokenizer2, lbl_encoder2)
            if(prediction==True):
                print('\nGroovy:\nThank you for your feedback, I am glad to hear that!')
            else:
                print('\nGroovy:\nI am sorry to hear that, I will feedback to my company.')
            print("Bye, take care"+ (newName if len(newName)!=0 else name))
        else:
            queryList[0] = queryList[1]
            queryList[1] = query.lower()
            
            result = model1.predict(pad_sequences(tokenizer1.texts_to_sequences([query]), truncating='post', maxlen=max_len), verbose=0) # remove the query from the end when its length exceed max length (20)
            tag = lbl_encoder1.inverse_transform([np.argmax(result)]) #transform tag's numerical values back to string (0 -> "greeting")

            for i in data1['intents']:
                if i['tag'] == tag:
                    response = np.random.choice(i['responses'])
                    if response == "Menu":
                        openMenu()                  
                        flag2 = True
                    elif response == "transaction":
                        flag2 = transaction(queryList, model2, tokenizer2, lbl_encoder2)
                    elif response == "username":
                        print("\nGroovy:\nYour name is" + (newName if len(newName)!= 0 else name) + ".")
                    elif response == "Change name":
                        newName = NER(query)
                        print("\nGroovy:\nNoted, your name is" + (newName if len(newName)!= 0 else name) + ".")
                    else:
                        print("\nGroovy:\n", response)

## Run machine-learning chatbot

In [17]:
# Run machine-learning chatbot
chat()

Groovy:
Hello, welcome to Chin's Coffee House! What is your name?



User:
 John



Groovy:
Hi John. Welcome to Chin's Coffee House. How can I help you? If you want to exit, type quit.



User:
 what is my name



Groovy:
Your name is John.

Groovy:
Hi John. Welcome to Chin's Coffee House. How can I help you? If you want to exit, type quit.


...


Thank you! Your order has been confirmed.

Groovy:
Hi John. Welcome to Chin's Coffee House. How can I help you? If you want to exit, type quit.



User:
 quit



Groovy:
Thank you for visiting Chin's Coffee House. What do you think of my service?



User:
 nice



Groovy:
Thank you for your feedback, I am glad to hear that!
Bye, take care John


# Experiments: Machine Learning Chatbot Evaluation

## Evaluation of intent matching with correct English

In [14]:
#Evaluation of intent matching with correct English
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

import csv
import statistics

k = 0
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

while(k < 10):
    # Read question-answer pair in csv file
    with open('ExperimentQuestions.csv', mode='r') as f:
        reader = csv.reader(f)
        corpus = list(reader)

    with open('intent.json', encoding='utf-8') as file:
        data1 = json.load(file)

    corpus = [sentence for sublist in corpus for sentence in sublist]
    prediction = []
    label = ["greeting", "greeting", "greeting", "greeting", "greeting", "greeting", "smalltalk", "formalgreeting", "formalgreeting", "help", "username", "chatbot", "chatbot",
             "help", "americano", "mocha", "coffee bean", "espresso price", "recommendation", "latte", "decaf","menu", "menu", "transaction", "transaction", "transaction", 
             "transaction", "transaction", "transaction", "changename", "changename", "changename", "changename", "changename", "changename"]

    max_len = 20

    model1, lbl_encoder1, tokenizer1 = model(data1)

    for sentence in corpus:
        query = sentence
        result = model1.predict(pad_sequences(tokenizer1.texts_to_sequences([query]), truncating='post', maxlen=max_len), verbose=0) # remove the query from the end when its length exceed max length (20)
        tag = lbl_encoder1.inverse_transform([np.argmax(result)]) #transform tag's numerical values back to string (0 -> "greeting")

        for i in data1['intents']:
            if i['tag'] == tag:
                prediction.append(tag[0])
    
    accuracy_list.append(accuracy_score(label, prediction))
    precision_list.append(precision_score(label, prediction, average = 'weighted', zero_division=1))
    recall_list.append(recall_score(label, prediction, average = 'weighted', zero_division=1))
    f1_list.append(f1_score(label, prediction, average='weighted'))
    k = k+1

    
#Print accuracy, precision, recall and f1 score
print("accurary: ", statistics.mean(accuracy_list))
print("f1_score: ", statistics.mean(f1_list))
print("precision: ", statistics.mean(precision_list))
print("recall: ", statistics.mean(recall_list))

accurary:  0.7628571428571429
f1_score:  0.7727137624280481
precision:  0.8848707482993197
recall:  0.7628571428571429


## Evaluation of intent matching with typo and grammatical error

In [27]:
#Evaluation of intent matching with misspelled words and incorrect grammar
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

import csv
import statistics

k = 0
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

while(k < 10):
    # Read question-answer pair in csv file
    with open('ExperimentQuestions_With_Incorrect_English.csv', mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        corpus = list(reader)

    with open('intent.json', encoding='utf-8') as file:
        data1 = json.load(file)

    corpus = [sentence for sublist in corpus for sentence in sublist]
    prediction = []
    label = ["greeting", "greeting", "greeting", "greeting", "greeting", "greeting", "smalltalk", "formalgreeting", "formalgreeting", "help", "username", "chatbot", 
             "chatbot", "help", "cappuccino", "latte", "coffee bean", "espresso price", "recommendation", "latte", "decaf","menu", "menu", "transaction", "transaction",
             "transaction", "transaction", "transaction", "transaction", "changename", "changename", "changename", "changename", "changename", "changename"]

    max_len = 20

    model1, lbl_encoder1, tokenizer1 = model(data1)

    for sentence in corpus:
        query = sentence
        result = model1.predict(pad_sequences(tokenizer1.texts_to_sequences([query]), truncating='post', maxlen=max_len), verbose=0) # remove the query from the end when its length exceed max length (20)
        tag = lbl_encoder1.inverse_transform([np.argmax(result)]) #transform tag's numerical values back to string (0 -> "greeting")

        for i in data1['intents']:
            if i['tag'] == tag:
                prediction.append(tag[0])

    accuracy_list.append(accuracy_score(label, prediction))
    precision_list.append(precision_score(label, prediction, average = 'weighted', zero_division=1))
    recall_list.append(recall_score(label, prediction, average = 'weighted', zero_division=1))
    f1_list.append(f1_score(label, prediction, average='weighted'))
    k = k+1
    
#Print accuracy, precision, recall and f1 score
print("accurary: ", statistics.mean(accuracy_list))
print("f1_score: ", statistics.mean(f1_list))
print("precision: ", statistics.mean(precision_list))
print("recall: ", statistics.mean(recall_list))

accurary:  0.6457142857142857
f1_score:  0.6447829313543599
precision:  0.8274557823129252
recall:  0.6457142857142857


## Evaluation of sentiment analysis for feedback

In [33]:
#Evaluation of sentiment analysis for feedback and user confirmation during beverage ordering
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

import statistics

k = 0
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

while(k < 10):
    with open('PositiveNegativeData.json', encoding='utf-8') as file1:
        data2 = json.load(file1)

    corpus = ["That was very helpful, thank you!",
              "I didn't quite understand your response.",
              "Your chatbot needs improvement in understanding natural language.",
              "Your chatbot is doing a great job at answering my questions!",
              "That was a quick and accurate response, good job!",
              "I'm not satisfied with your chatbot's response, can you provide more information?",
              "Your chatbot is very informative and easy to use.",
              "I think your chatbot misunderstood my question, can you clarify?",
              "Your chatbot is well-performed, thanks!",
              "The chatbot's response was a bit too vague, can you provide more detail?",
              "good",
              "bad", 
              "I really enjoyed using your chatbot to order my drink. It was quick and easy to navigate.", 
              "Your chatbot was able to accurately understand my order and provide helpful suggestions. Great job!", 
              "I had a bit of trouble with the chatbot understanding my order. Maybe adding more prompts or options could help.", 
              "The chatbot seemed a bit slow and unresponsive at times, which was frustrating. Maybe there's a way to speed up the system?", 
              "The chatbot didn't seem to have all the drink options I was looking for. It would be great to see a wider selection in the future.",
              "I appreciate that your chatbot gave me the option to customize my drink order. It made the experience feel more personalized.", 
              "The chatbot gave me an error message when trying to process my payment. It would be helpful to have clearer instructions or troubleshooting tips.", 
              "Overall, I had a positive experience using your chatbot to order my beverage. Thank you for making the process more convenient!"]

    label = ["positive", "negative", "negative", "positive", "positive", "negative", "positive", "negative", "positive", "negative", "positive", "negative", "positive", "positive",
             "negative", "negative", "negative", "positive", "negative", "positive"]

    prediction = []

    model2, lbl_encoder2, tokenizer2 = model(data2)

    for query in corpus:
        flag = feedback(query, model2, tokenizer2, lbl_encoder2)

        if(flag==True):
            prediction.append("positive")
        else:
            prediction.append("negative")

    accuracy_list.append(accuracy_score(label, prediction))
    precision_list.append(precision_score(label, prediction, average = 'weighted', zero_division=1))
    recall_list.append(recall_score(label, prediction, average = 'weighted', zero_division=1))
    f1_list.append(f1_score(label, prediction, average='weighted'))
    k = k+1
    
#Print accuracy, precision, recall and f1 score
print("accurary: ", statistics.mean(accuracy_list))
print("f1_score: ", statistics.mean(f1_list))
print("precision: ", statistics.mean(precision_list))
print("recall: ", statistics.mean(recall_list))

accurary:  0.605
f1_score:  0.5976233245673278
precision:  0.6120751470751471
recall:  0.605


## Evaluation of intent matching when having contextual queries

In [34]:
# Accuracy of intent matching when user refer to what they talked before
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
import csv
import statistics

k = 0
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

while(k < 10):  
    with open('intent.json', encoding='utf-8') as file:
        data1 = json.load(file)

    corpus = ["What is Americano", "How much is it", 
              "How much is decaf coffee", "Can i have one of it", 
              "What is Mocha?", "Can I have one of it",
              "What is flat white", "Please give me one of it and one black tea", 
              "My name is Hong Shen", "What is my name"]

    label = ["americano price", "transaction", "transaction", "transaction", "username"]

    prediction = []
    predicted = []
    max_len = 20

    model1, lbl_encoder1, tokenizer1 = model(data1)

    for sentence in corpus:
        query = sentence
        result = model1.predict(pad_sequences(tokenizer1.texts_to_sequences([query]), truncating='post', maxlen=max_len), verbose=0) # remove the query from the end when its length exceed max length (20)
        tag = lbl_encoder1.inverse_transform([np.argmax(result)]) #transform tag's numerical values back to string (0 -> "greeting")

        for i in data1['intents']:
            if i['tag'] == tag:
                predicted.append(tag[0])

    prediction = [predicted[i] for i in range(1, len(predicted), 2)]

    accuracy_list.append(accuracy_score(label, prediction))
    precision_list.append(precision_score(label, prediction, average = 'weighted', zero_division=1))
    recall_list.append(recall_score(label, prediction, average = 'weighted', zero_division=1))
    f1_list.append(f1_score(label, prediction, average='weighted'))
    k = k+1
    
#Print accuracy, precision, recall and f1 score
print("accurary: ", statistics.mean(accuracy_list))
print("f1_score: ", statistics.mean(f1_list))
print("precision: ", statistics.mean(precision_list))
print("recall: ", statistics.mean(recall_list))
    

accurary:  0.8200000000000001
f1_score:  0.8200000000000001
precision:  1.0
recall:  0.8200000000000001


## Average chatbot response time

In [22]:
# Calculate avergae chatbot response time
import time
import statistics
import csv
# Read question-answer pair in csv file
with open('ExperimentQuestions.csv', mode='r') as f:
    reader = csv.reader(f)
    corpus = list(reader)
    
with open('intent.json', encoding='utf-8') as file:
    data1 = json.load(file)
    
with open('PositiveNegativeData.json', encoding='utf-8') as file1:
    data2 = json.load(file1)
        
corpus = [sentence for sublist in corpus for sentence in sublist]

max_len = 20

model1, lbl_encoder1, tokenizer1 = model(data1)
model2, lbl_encoder2, tokenizer2 = model(data2)

name = ''
newName = ''
queryList = ["",""]
response_time = []
for sentence in corpus:
    query = sentence
    queryList[0] = queryList[1]
    queryList[1] = query.lower()
    start_time = time.time()
    result = model1.predict(pad_sequences(tokenizer1.texts_to_sequences([query]), truncating='post', maxlen=max_len), verbose=0) # remove the query from the end when its length exceed max length (20)
    tag = lbl_encoder1.inverse_transform([np.argmax(result)]) #transform tag's numerical values back to string (0 -> "greeting")

    for i in data1['intents']:
        if i['tag'] == tag:
            response = np.random.choice(i['responses'])
            if response == "Menu":
                openMenu()                  
            elif response == "transaction":
                print("Please confirm your order")
                print("Thank you, your order has been confirmed")
            elif response == "username":
                print("\nGroovy:\nYour name is" + (newName if len(newName)!= 0 else name) + ".")
            elif response == "Change name":
                newName = NER(query)
                print("\nGroovy:\nNoted, your name is" + (newName if len(newName)!= 0 else name) + ".")
            else:
                print("\nGroovy:\n", response)
            end_time = time.time()
            response_time.append(round(end_time - start_time, 4)) # calculate response time
            print("Response time: ", response_time[-1])
print(response_time)

# sentiment analysis response time
corpus = ["That was very helpful, thank you!",
          "I didn't quite understand your response.",
          "Your chatbot needs improvement in understanding natural language.",
          "Your chatbot is doing a great job at answering my questions!",
          "That was a quick and accurate response, good job!",
          "I'm not satisfied with your chatbot's response, can you provide more information?",
          "Your chatbot is very informative and easy to use.",
          "I think your chatbot misunderstood my question, can you clarify?",
          "good",
          "bad"]

for query in corpus:
    start_time = time.time()
    prediction = feedback(query, model2, tokenizer2, lbl_encoder2)
    if(prediction==True):
        print('\nGroovy:\nThank you for your feedback, I am glad to hear that!')
    else:
        print('\nGroovy:\nI am sorry to hear that, I will feedback to my company.')
        print("Bye, take care"+ (newName if len(newName)!=0 else name))
    end_time = time.time()
    response_time.append(round(end_time - start_time, 4)) # calculate response time
    print("Response time: ", response_time[-1])

print("\nTotal response: ",len(response_time))
print("\n",response_time)

# calculate average response time
avg_response_time = statistics.mean(response_time)
print("\nAverage response time: ", avg_response_time)



Groovy:
 Hi there
Response time:  0.3037

Groovy:
 Hi there
Response time:  0.0377

Groovy:
 Hi there
Response time:  0.0332

Groovy:
 Hey
Response time:  0.0406

Groovy:
 Hi
Response time:  0.0349

Groovy:
 I'm Groovy, an Artificial Intelligent bot
Response time:  0.0306

Groovy:
 I am fine, thanks for asking.
Response time:  0.0455

Groovy:
 Good day!
Response time:  0.0303

Groovy:
 Good day!
Response time:  0.0402

Groovy:
 I can have a small talk, help to take order and answer some questions regarding the beverages!
Response time:  0.0402

Groovy:
Your name is.
Response time:  0.0396

Groovy:
 I'm Groovy, an Artificial Intelligent bot
Response time:  0.0347

Groovy:
 I.m Groovy, your bot assistant
Response time:  0.0428


Menu:

---------------------------------

Espresso		(£1.50)

---------------------------------

Americano		(£1.50)

---------------------------------

Cappuccino		(£1.80)

---------------------------------

Latte			(£1.80)

---------------------------------

Mac